In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.2
scikit-learn-intelex==2021.20220215.212715


In [2]:
!python -V

Python 3.9.12


In [3]:
import pickle
import pandas as pd

/home/cloud_user/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/cloud_user/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [6]:
with open('models/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/cloud_user/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/cloud_user/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
y_pred.std()

6.247488852238703

Q1: 6.24

In [13]:
df['ride_id'] = f'2023/03_' + df.index.astype('str')

In [19]:
df_q2 = pd.DataFrame()
df_q2['ride_id'] = df['ride_id']
df_q2['predicted_duration'] = y_pred
df_q2.to_parquet(
    'q2_output_file',
    engine='pyarrow',
    compression=None,
    index=False
)

In [20]:
!ls -lh q2_output_file

-rw-rw-r-- 1 cloud_user cloud_user 66M Oct 27 09:57 q2_output_file


Q2: 66M

In [21]:
!jupyter nbconvert --to script 'Homework #4.ipynb'

[NbConvertApp] Converting notebook Homework #4.ipynb to script
[NbConvertApp] Writing 1405 bytes to Homework #4.py


Q4: jupyter nbconvert --to script 'Homework #4.ipynb'

In [22]:
!cat Pipfile.lock | jq '.default."scikit-learn".hashes[0]'

"sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"


Q3: "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"

In [26]:
!python3 'Homework #4.py' '2023' '04' 'models/model.bin'

/home/cloud_user/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/cloud_user/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/home/cloud_user/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/cloud_user/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarnin

Q5: 14.29

In [27]:
!docker run homework_4 '2023' '05' '/app/model.bin'

0.19174419265916945


Q6: 0.19